In [1]:
import pandas as pd
import numpy as np 
import json


In [2]:
from google.colab import files
uploaded = files.upload()

Saving __ (1).ipynb to __ (1).ipynb
Saving train.jsonl to train.jsonl
Saving val_wtihout.jsonl to val_wtihout.jsonl


In [4]:
def get_X_y(data_json_file):
    X, y = [], []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    X.append(text+' Query: '+query+' Answer: '+a['text'])
                    y.append(a['label'])
    return X, y

X_train, y_train = get_X_y('train.jsonl')

train_df = pd.DataFrame({
    'text': X_train,
    'label': y_train
})
train_df

,text,label
0,"(1) Но люди не могут существовать без природы,...",1
1,"(1) Но люди не могут существовать без природы,...",0
2,"(1) Но люди не могут существовать без природы,...",0
3,"(1) Но люди не могут существовать без природы,...",1
4,"(1) Но люди не могут существовать без природы,...",0
...,...,...
11945,(1) Известный американский боец смешанных един...,0
11946,(1) Известный американский боец смешанных един...,1
11947,(1) Известный американский боец смешанных един...,0
11948,(1) Известный американский боец смешанных един...,0


In [6]:
def get_X_test(data_json_file):
    X1 = []
    X2 = []
    X3 = []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    X1.append(text) #текст
                    X2.append(query) #вопрос
                    X3.append(a['text']) #ответ на вопрос
    return X1, X2, X3

X1, X2, X3 = get_X_test('val_wtihout.jsonl')

test = pd.DataFrame({
    'text': X1,
    'query': X2,
    'answer': X3
})

test

,text,query,answer
0,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,Так перевозили особо важных заключенных.
1,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что был эмигрантом."
2,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он сам вырыл себе землянку."
3,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он побывал на пересылке Красная Пр..."
4,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он был особо важным заключённым и ..."
...,...,...,...
2230,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Изнасилование.
2231,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Рокко похитил любовника и присвоил его деньги.
2232,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Убийство и расчленение тела.
2233,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Рокко убил любовника и расчленил его тело.


In [7]:
punctuation_signs = list(":!.»«,:;)(")
for punct_sign in punctuation_signs:
    test['text'] = test['text'].str.replace(punct_sign, '')

test['text'] = test['text'].str.lower()
test['text'] = test['text'].str.replace('"', '')
test['text'] = test['text'].str.replace("\\n", " ") 

test['query'] = test['query'].str.lower()
test['query'] = test['query'].str.replace('"', '')
test['query'] = test['query'].str.replace("\\n", " ") 

test['answer'] = test['answer'].str.lower()
test['answer'] = test['answer'].str.replace('"', '')
test['answer'] = test['answer'].str.replace("\\n", " ")

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df['text'], train_df['label'], test_size=0.33, random_state=42)

vectorizer = CountVectorizer(ngram_range=(3,6), analyzer='word', max_features = 10000)

X_train_vector = vectorizer.fit_transform(X_train).toarray()
X_test_vector = vectorizer.transform(X_test).toarray()

In [10]:
print(f'X train shape: {X_train.shape}')
print(f'X test shape: {X_test.shape}')

X train shape: (8006,)
X test shape: (3944,)


In [11]:
logr = LogisticRegression(random_state = 42)   
randf = RandomForestClassifier(random_state = 42, n_jobs =-1)
dect = DecisionTreeClassifier(random_state = 42)

models = [logr, randf, dect]

for model in models:
    print(model)
    model.fit(X_train_vector, y_train)
    predictions = model.predict(X_test_vector)
    print(classification_report(y_test, predictions))
    print('=' * 50)

LogisticRegression(random_state=42)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0       0.52      0.79      0.62      2127
           1       0.37      0.15      0.21      1817

    accuracy                           0.49      3944
   macro avg       0.44      0.47      0.42      3944
weighted avg       0.45      0.49      0.43      3944

RandomForestClassifier(n_jobs=-1, random_state=42)
              precision    recall  f1-score   support

           0       0.51      0.72      0.60      2127
           1       0.36      0.18      0.24      1817

    accuracy                           0.47      3944
   macro avg       0.43      0.45      0.42      3944
weighted avg       0.44      0.47      0.43      3944

DecisionTreeClassifier(random_state=42)
              precision    recall  f1-score   support

           0       0.51      0.77      0.62      2127
           1       0.35      0.14      0.20      1817

    accuracy                           0.48      3944
   macro avg       0.43      0.46    

In [12]:
test_values = test["text"].tolist()
result = vectorizer.transform(test_values)
logr_predictions = logr.predict(result)

In [13]:
answ = pd.DataFrame(logr_predictions, columns=['label'])
answ.reset_index(inplace=True)
answ.shape

(2235, 2)

In [14]:
answ.to_csv('Melly.csv', index=False)
files.download("Melly.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>